In [1]:
import pandas as pd

In [2]:
def process_inactive_rows(df):
    '''
        폐업 및 삭제된 시설에 관한 데이터 제거
    '''
    _df = df.copy()
    if 'FCLTY_STATE_VALUE' in _df.columns:
        _df = _df[_df['FCLTY_STATE_VALUE'] != '폐업']
    if 'DEL_AT' in _df.columns:
        _df = _df[_df['DEL_AT'] != 'Y']
    return _df

def process_unnecessary_cols(df):
    '''
     불필요한 컬럼을 제거함
    '''
    _df = df.copy()
    drop_cols = [
        'FCLTY_MANAGE_LI_CD',
        'FCLTY_MANAGE_LI_NM',
        'ERDSGN_AT',
        'ATNM_CHCK_TRGET_AT',
        'DATA_ORIGIN_FLAG_CD',
        'DEL_AT',
    ]
    drop_cols = [i for i in drop_cols if i in _df.columns]
    _df.drop(columns=drop_cols, inplace=True)
    return _df


def process_duplicate_rows(df):
    '''
        시설명, 소재지, 시설유형명이 같은 데이터는 동일 시설로 간주하고 중복 제거
    '''
    _df = df.copy()
    def first_non_null(series):
        return series.dropna().iloc[0] if not series.dropna().empty else None
    _df = _df.groupby(
        ["FCLTY_NM", "POSESN_MBY_CTPRVN_CD", "INDUTY_NM"], as_index=False
    ).agg(first_non_null)
    return _df


In [3]:
public_facility_status = pd.read_csv('./data/전국공공체육시설 데이터.csv',
                                     low_memory=False)
public_facility_status = process_inactive_rows(public_facility_status)
public_facility_status = process_unnecessary_cols(public_facility_status)
public_facility_status = process_duplicate_rows(public_facility_status)
public_facility_status.to_csv('./data/전국공공체육시설 데이터_cleaned.csv', index=False)
public_facility_status.shape


(32112, 32)

In [4]:
all_facility_status = pd.read_csv('./data/전국체육시설현황 데이터.csv',
                              on_bad_lines='skip',
                              low_memory=False)
# '폐업' 상태인 행 드롭
all_facility_status = process_inactive_rows(all_facility_status)
all_facility_status = process_unnecessary_cols(all_facility_status)
all_facility_status = process_duplicate_rows(all_facility_status)
all_facility_status.to_csv('./data/전국체육시설현황 데이터_cleaned.csv', index=False)
all_facility_status.shape

(52716, 52)

In [5]:
all_facility_status['FCLTY_NM'].nunique()

49624

In [6]:
# 최종적으로 두 데이터프레임을 병합하고 결과값을 csv로 저장

merged_df = pd.merge(
    all_facility_status,
    public_facility_status,
    on=['FCLTY_NM', 'POSESN_MBY_CTPRVN_CD', 'INDUTY_NM'],
    how='outer',
    suffixes=('', '_public')
)

# 중복 컬럼 처리 - all_facility_status의 데이터 우선 사용
for col in merged_df.columns:
    if col.endswith('_public'):
        original_col = col[:-7]  # '_public' 제거
        # null 값인 경우에만 public_facility의 값으로 대체
        merged_df[original_col] = merged_df[original_col].fillna(merged_df[col])
        merged_df.drop(columns=[col], inplace=True)

merged_df.to_csv('./data/merged_facility_status.csv', index=False)
merged_df.shape


(53570, 63)